In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

In [1]:
# files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mertbayraktar/belka-enc

!unzip \*.zip -d ./data/

Dataset URL: https://www.kaggle.com/datasets/mertbayraktar/belka-enc
License(s): unknown
 91% 55.0M/60.5M [00:00<00:00, 110MB/s] 
100% 60.5M/60.5M [00:00<00:00, 88.3MB/s]
Archive:  belka-enc.zip
  inflating: ./data/test_enc.parquet  
  inflating: ./data/train_enc.parquet  


In [ ]:
!ls ./data/

test_enc.parquet  train_enc.parquet


In [ ]:
!pip install fastparquet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.9 MB/s eta 0:00:00


In [ ]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [ ]:
train = pd.read_parquet('/content/data/train_enc.parquet')
test = pd.read_parquet('/content/data/test_enc.parquet')
tst = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/test.parquet')

In [ ]:
pd.options.display.max_rows = 250
pd.options.display.max_columns = 250

In [ ]:
train.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,binds_BRD4,binds_HSA,binds_sEH,MF,MW,Ncharges,C,H,N,O,Cl,F,S,Br,I,B,Si,RNH2,R2NH,R3N,ROH,RCHO,RCOR,RCOOH,RCOOR,ROR,RCCH,RCN,RINGS,AROMATIC,MF_block2,MW_block2,Ncharges_block2,C_block2,H_block2,N_block2,O_block2,Cl_block2,F_block2,S_block2,Br_block2,I_block2,B_block2,Si_block2,RNH2_block2,R2NH_block2,R3N_block2,ROH_block2,RCHO_block2,RCOR_block2,RCOOH_block2,RCOOR_block2,ROR_block2,RCCH_block2,RCN_block2,RINGS_block2,AROMATIC_block2,MF_block3,MW_block3,Ncharges_block3,C_block3,H_block3,N_block3,O_block3,Cl_block3,F_block3,S_block3,Br_block3,I_block3,B_block3,Si_block3,RNH2_block3,R2NH_block3,R3N_block3,ROH_block3,RCHO_block3,RCOR_block3,RCOOH_block3,RCOOR_block3,ROR_block3,RCCH_block3,RCN_block3,RINGS_block3,AROMATIC_block3
0,0,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
1,1,0,0,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
2,2,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
3,3,0,0,0,0,0,C21H19NO4,349.37986,0,21,19,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1
4,4,0,0,0,0,0,C20H17NO4,335.35328,0,20,17,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,6,2,C10H11ClNO,196.65344,0,10,11,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,C9H14Br2N4,338.04226,0,9,14,4,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98415610 entries, 0 to 98415609
Data columns (total 87 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   buildingblock1_smiles  int64  
 1   buildingblock2_smiles  int64  
 2   buildingblock3_smiles  int64  
 3   binds_BRD4             int8   
 4   binds_HSA              int8   
 5   binds_sEH              int8   
 6   MF                     object 
 7   MW                     float64
 8   Ncharges               int64  
 9   C                      int64  
 10  H                      int64  
 11  N                      int64  
 12  O                      int64  
 13  Cl                     int64  
 14  F                      int64  
 15  S                      int64  
 16  Br                     int64  
 17  I                      int64  
 18  B                      int64  
 19  Si                     int64  
 20  RNH2                   int64  
 21  R2NH                   int64  
 22  R3N             

In [ ]:
# Memory usage of the entire DataFrame
print(train.memory_usage(deep=True))

# Total memory usage of the DataFrame
print(f'Total memory usage: {train.memory_usage(deep=True).sum()} bytes')

Index                           128
buildingblock1_smiles     787324880
buildingblock2_smiles     787324880
buildingblock3_smiles     787324880
binds_BRD4                 98415610
binds_HSA                  98415610
binds_sEH                  98415610
MF                       6600390644
MW                        787324880
Ncharges                  787324880
C                         787324880
H                         787324880
N                         787324880
O                         787324880
Cl                        787324880
F                         787324880
S                         787324880
Br                        787324880
I                         787324880
B                         787324880
Si                        787324880
RNH2                      787324880
R2NH                      787324880
R3N                       787324880
ROH                       787324880
RCHO                      787324880
RCOR                      787324880
RCOOH                     78

In [ ]:
def downcast_int_columns(df):
    int_columns = df.select_dtypes(include=['int64']).columns
    for col in int_columns:
        min_val = df[col].min()
        max_val = df[col].max()
        if -128 <= min_val <= max_val <= 127:
            df[col] = df[col].astype('int8')
            print(f"Column {col} downcast to int8")
        elif -32768 <= min_val <= max_val <= 32767:
            df[col] = df[col].astype('int16')
            print(f"Column {col} downcast to int16")
        else:
            print(f"Column {col} cannot be downcast to int8 or int16 due to value range.")
    return df

# Apply the function to your DataFrame
train = downcast_int_columns(train)
test = downcast_int_columns(test)

# Check the memory usage after downcasting
print(train.memory_usage(deep=True))

Column buildingblock1_smiles downcast to int16
Column buildingblock2_smiles downcast to int16
Column buildingblock3_smiles downcast to int16
Column Ncharges downcast to int8
Column C downcast to int8
Column H downcast to int8
Column N downcast to int8
Column O downcast to int8
Column Cl downcast to int8
Column F downcast to int8
Column S downcast to int8
Column Br downcast to int8
Column I downcast to int8
Column B downcast to int8
Column Si downcast to int8
Column RNH2 downcast to int8
Column R2NH downcast to int8
Column R3N downcast to int8
Column ROH downcast to int8
Column RCHO downcast to int8
Column RCOR downcast to int8
Column RCOOH downcast to int8
Column RCOOR downcast to int8
Column ROR downcast to int8
Column RCCH downcast to int8
Column RCN downcast to int8
Column RINGS downcast to int8
Column AROMATIC downcast to int8
Column Ncharges_block2 downcast to int8
Column C_block2 downcast to int8
Column H_block2 downcast to int8
Column N_block2 downcast to int8
Column O_block2 do

In [ ]:
def downcast_float_columns(df):
    float_columns = df.select_dtypes(include=['float64']).columns
    for col in float_columns:
        df[col] = df[col].astype('float32')
        print(f"Column {col} downcast to float32")
    return df

# Apply the function to your DataFrame
train = downcast_float_columns(train)
test = downcast_float_columns(test)


Column MW downcast to float32
Column MW_block2 downcast to float32
Column MW_block3 downcast to float32
Column MW downcast to float32
Column MW_block2 downcast to float32
Column MW_block3 downcast to float32


In [ ]:
# Memory usage of the entire DataFrame
print(train.memory_usage(deep=True))

# Total memory usage of the DataFrame
print(f'Total memory usage: {train.memory_usage(deep=True).sum()} bytes')

Index                           128
buildingblock1_smiles     196831220
buildingblock2_smiles     196831220
buildingblock3_smiles     196831220
binds_BRD4                 98415610
binds_HSA                  98415610
binds_sEH                  98415610
MF                       6600390644
MW                        393662440
Ncharges                   98415610
C                          98415610
H                          98415610
N                          98415610
O                          98415610
Cl                         98415610
F                          98415610
S                          98415610
Br                         98415610
I                          98415610
B                          98415610
Si                         98415610
RNH2                       98415610
R2NH                       98415610
R3N                        98415610
ROH                        98415610
RCHO                       98415610
RCOR                       98415610
RCOOH                      9

In [ ]:
# Identify object columns
object_columns = train.select_dtypes(include=['object']).columns

# Drop object columns
train = train.drop(columns=object_columns)


object_test = test.select_dtypes(include=['object']).columns

# Drop object columns
test = test.drop(columns=object_test)



In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98415610 entries, 0 to 98415609
Data columns (total 84 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   buildingblock1_smiles  int16  
 1   buildingblock2_smiles  int16  
 2   buildingblock3_smiles  int16  
 3   binds_BRD4             int8   
 4   binds_HSA              int8   
 5   binds_sEH              int8   
 6   MW                     float32
 7   Ncharges               int8   
 8   C                      int8   
 9   H                      int8   
 10  N                      int8   
 11  O                      int8   
 12  Cl                     int8   
 13  F                      int8   
 14  S                      int8   
 15  Br                     int8   
 16  I                      int8   
 17  B                      int8   
 18  Si                     int8   
 19  RNH2                   int8   
 20  R2NH                   int8   
 21  R3N                    int8   
 22  ROH             

In [ ]:
train.to_parquet('train_enc_dc.parquet')
test.to_parquet('test_enc_dc.parquet')

In [ ]:
train.shape, test.shape

((98415610, 84), (1674896, 84))